In [17]:
import numpy as np
import json_tricks


inputs = json_tricks.load('inputs.json')
answer = {}


# Task

You are given a linear hyperspace by 2 entities:
- a vector that belongs to the hyperspace $\mathbf x_0$
- a normal vector to this hyperspace $\mathbf w$

So that the hyperplane is given by equation:

$\left<\mathbf w, \mathbf x - \mathbf x_0\right> = 0$

You are also given another vector $\mathbf y$. 

Find distance from this vector to the given hyperplane.

In [18]:
def distance_to_hyperplane_1(y, w, x0):
    res = 0
    w = np.array(w)
    y = np.array(y)
    x0 = np.array(x0)
    numerator = np.abs(np.dot(w, y - x0))
    denominator = np.linalg.norm(w)
    res = numerator / denominator
    return res

In [19]:
answer['task1'] = []
for one_input in inputs['task1']:
    answer['task1'].append(
        distance_to_hyperplane_1(**one_input))

# Task

You are given a linear hyperspace by 2 entities:
- a vector that belongs to the hyperspace $\mathbf x_0$
- a set of direction vectors for this hyperspace $\mathbf a_1, \dots, \mathbf a_K$ in form of column-matrix 
    
    $A = \begin{bmatrix}
    | & | & & | \\
    \mathbf a_1 & \mathbf a_2 & \dots & \mathbf a_K \\
    | & | & & | \\
    \end{bmatrix}$

So that the hyperplane is given by equation:

$\mathbf x_0 + \sum_{k=1}^K \alpha_k \mathbf a_k = \mathbf 0$

You are also given another vector $\mathbf y$. 

Find distance from this vector to the given hyperplane.

In [33]:
def distance_to_hyperplane_2(y, A, x0):
    y = np.array(y).flatten()
    x0 = np.array(x0).flatten()
    A = np.array(A)
    
    if A.size == 0:
        return np.linalg.norm(y - x0)
    
    try:
        AT = A.T
        ATA = AT @ A
        ATA_inv = np.linalg.inv(ATA)
        P = A @ ATA_inv @ AT
        
        diff = y - x0
        perp = diff - P @ diff
        
        return np.linalg.norm(perp)
    except np.linalg.LinAlgError:
        diff = y - x0
        alpha = np.linalg.lstsq(A, diff, rcond=None)[0]
        proj = A @ alpha
        return np.linalg.norm(diff - proj)

In [34]:
answer['task2'] = []
for one_input in inputs['task2']:
    answer['task2'].append(
        distance_to_hyperplane_2(**one_input))

# Task

You are given 2 linear hyperspaces, each defined by 2 entities:
- a vector that belongs to the hyperspace $\mathbf x_0$
- a set of direction vectors for this hyperspace $\mathbf a_1, \dots, \mathbf a_K$ in form of column-matrix 

So that each of the hyperplanes is given by equation:

$\mathbf x_0 + \sum_{k=1}^K \alpha_k \mathbf a_k = \mathbf 0$

Find distance between these hyperplanes.

In [ ]:
def distance_to_hyperplane_3(A, x0, B, y0):
    A = np.array(A)
    x0 = np.array(x0)
    B = np.array(B)
    y0 = np.array(y0)
    # If the subspaces intersect, the distance is zero
    # Otherwise, find the minimal distance between points in the two affine subspaces
    if A.size == 0 and B.size == 0:
        # Both are points
        res = np.linalg.norm(x0 - y0)
    else:
        # We want to minimize ||(x0 + A @ alpha) - (y0 + B @ beta)|| over alpha, beta
        # This is a standard least squares problem:
        # Let z = [alpha; beta], M = [A, -B], c = y0 - x0
        # Minimize ||A*alpha - B*beta - (y0 - x0)|| = ||M @ z - c||
        M = np.hstack([A, -B]) if (A.size > 0 and B.size > 0) else (A if A.size > 0 else -B)
        c = y0 - x0
        if M.size == 0:
            # Both are points
            perp = c
        else:
            z_hat = np.linalg.lstsq(M, c, rcond=None)[0]
            proj = M @ z_hat
            perp = c - proj
        res = np.linalg.norm(perp)
    return res

In [ ]:
answer['task3'] = []
for one_input in inputs['task3']:
    answer['task3'].append(
        distance_to_hyperplane_3(**one_input))

In [ ]:
json_tricks.dump(answer, '.answer.json')

'{"task1": [1.1611015532036637, 1.2169686348578783, 3.8171960993534175, 2.1577367083828145, 17.53449972356085, 13.749230523405476, 0.11441551070947108, 0.9750002110024923, 2.0481280123278074, 0.4178554470186725, 7.299732493572025, 5.405127802564358, 2.0541374149458744, 6.71079347281296, 3.03653640842388, 9.498185286556286, 3.7630890450319083, 1.8641906049463526, 10.670770020027398, 5.291772589866002, 0.912245460839306, 0.20732586478679277, 12.816310409006174, 1.6373225835853016, 6.291571550369559, 20.239709319388293, 16.466083091274903, 13.474719001142073, 5.2761044442380856, 0.7250110520819842, 4.519523979103389, 4.225904366704887, 4.775521858228876, 13.912436341952132, 7.24644943484495, 9.313787360469487, 1.0596535411727601, 5.682048493486052, 4.5807379490262905, 3.3300869947532497, 5.307227776030219, 5.6272294256742645, 4.913975701062781, 4.802205375719392, 6.501355432613329, 11.47165060247919, 1.1319709719592113, 4.803995448402746, 3.272807297357087, 15.777869833110813, 4.517699029